# Laboratorio 2
Este laboratorio se divide dentro de 3 tasks diferentes. El primer task se encuentra dentro del archivo Informe.pdf, dónde se respondieron preguntas teóricas sobre diferentes modelos / algoritmos aplicados en situaciones de la vida real.

Para los siguientes dos tasks, primero debemos descargar un dataset y limpiar los datos. Luego, debemos implementar algoritmos de ML desde 0 para aplicarlos sobre los datos limpios.

## Task 2

### 1. Carga y Limpieza
Primero, vamos a utilizar Kagglehub para poder descargar el dataset. De esta manera, al correr el Notebook en algún otro dispositivo no debemos estar manejando la transferencia del archivo .csv

In [6]:
import kagglehub
import pandas as pd

# Download dataset and get path to directory
path = kagglehub.dataset_download("shashwatwork/web-page-phishing-detection-dataset")

# Read csv at path
df = pd.read_csv(f"{path}/dataset_phishing.csv")

# Show DF
df.head(20)

,url,length_url,length_hostname,ip,nb_dots,nb_hyphens,nb_at,nb_qm,nb_and,nb_or,...,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank,status
0,http://www.crestonwood.com/router.php,37,19,0,3,0,0,0,0,0,...,0,1,0,45,-1,0,1,1,4,legitimate
1,http://shadetreetechnology.com/V4/validation/a...,77,23,1,1,0,0,0,0,0,...,1,0,0,77,5767,0,0,1,2,phishing
2,https://support-appleld.com.secureupdate.duila...,126,50,1,4,1,0,1,2,0,...,1,0,0,14,4004,5828815,0,1,0,phishing
3,http://rgipt.ac.in,18,11,0,2,0,0,0,0,0,...,1,0,0,62,-1,107721,0,0,3,legitimate
4,http://www.iracing.com/tracks/gateway-motorspo...,55,15,0,2,2,0,0,0,0,...,0,1,0,224,8175,8725,0,0,6,legitimate
5,http://appleid.apple.com-app.es/,32,24,0,3,1,0,0,0,0,...,1,1,1,0,-1,0,0,1,0,phishing
6,http://www.mutuo.it,19,12,0,2,0,0,0,0,0,...,0,1,0,170,7529,0,0,0,1,legitimate
7,http://www.shadetreetechnology.com/V4/validati...,81,27,1,2,0,0,0,0,0,...,1,0,0,76,5767,0,0,1,2,phishing
8,http://vamoaestudiarmedicina.blogspot.com/,42,34,0,2,0,0,0,0,0,...,1,1,0,371,7298,0,0,0,5,legitimate
9,https://parade.com/425836/joshwigler/the-amazi...,104,10,0,1,10,0,0,0,0,...,1,0,0,128,9368,6774,0,0,5,legitimate


Luego de haber cargado exitosamente el dataset, lo primero que podemos observar es que tenemos 89 columnas. Sería ideal si tuviéramos alguna documentación sobre el significado de cada una, sin embargo, la descripción oficial del dataset es la siguiente:

"The provided dataset includes 11430 URLs with 87 extracted features. The dataset is designed to be used as benchmarks for machine learning-based phishing detection systems. Features are from three different classes: 56 extracted from the structure and syntax of URLs, 24 extracted from the content of their correspondent pages, and 7 are extracted by querying external services. The dataset is balanced, it contains exactly 50% phishing and 50% legitimate URLs."

Para el scope de este laboratorio, únicamente debemos indagar sobre las variables más correlacionadas con la variable objetivo. Por lo tanto, vamos a iniciar encodeando la variable objetivo.

In [7]:
# Map text labels to binary representation
df['status'] = df['status'].map({'phishing': 1, 'legitimate': 0})

status_counts = df['status'].value_counts().reset_index()

status_counts.columns = ['Status Label', 'Total Count']

# Sanity check
status_counts

,Status Label,Total Count
0,0,5715
1,1,5715


Estos valores concuerdan con la descripción, dónde el dataset se encuentra perfectamente balanceado.

Ahora, podemos remover la columna de url, ya que tenemos una gran cantidad de atributos derivados numéricos. Luego, podemos identificar que columnas no tienen tipos de datos numéricos en el dataset restante.

In [ ]:
# Drop URL col
df = df.drop(columns=['url'])

# List non-numeric cols
no_num = df.select_dtypes(exclude=['number']).columns.tolist()

# Print list
print("Columnas no numéricas:")
print(no_num)

Columnas no numéricas:
[]


## 2. Selección de Features

Luego de verificar que todas las columnas sean numéricas, podemos calcular la correlación con la variable objetivo.

In [15]:
# Top 5 most correlated features with respect to target variable
top = df.corr()['status'].abs().sort_values(ascending=False)[1:6]

top.to_frame('Correlation')

,Correlation
google_index,0.731171
page_rank,0.511137
nb_www,0.443468
ratio_digits_url,0.356395
domain_in_title,0.342807


Podemos ver que las dos variables más correlacionadas claramente son `google_index` y `page_rank`, por lo cuál serán las variables con las que trabajaremos.

##